In [ ]:
# Importing the necessary libraries
import http.client
import datetime
import json
import pandas as pd
import os

In [ ]:
# Variables
days = 30 # Day limit: the code will browse for CTFs from today until the given limit 
restriction_list = ['Open', 'Individual', 'High-school'] # Filter the CTF you're interested in depending on the restriction types
useless_columns = ['ctf_id','weight', 'logo', 'live_feed', 'format', 'participants', 'public_votable', 'is_votable_now', 'prizes', 'organizers', 'format_id', 'duration', 'onsite', 'location', 'ctftime_url', 'restrictions'] # Columns you don't need for your integration
artifact_location = "\\artifacts\\events.xlsx"

In [ ]:
def get_future_events(day_limit):
    # Get the current date and time
    now = datetime.datetime.now(datetime.UTC)

    # Calculate the start and finish timestamps
    start_timestamp = int(now.timestamp())
    finish_timestamp = int((now + datetime.timedelta(days=day_limit)).replace(tzinfo=datetime.timezone.utc).timestamp())

    # Set the API endpoint and parameters
    url = '/api/v1/events/'
    params = {
        'limit': 500,
        'start': start_timestamp,
        'finish': finish_timestamp
    }

    # Build the query string
    query_string = '?' + '&'.join([f'{key}={value}' for key, value in params.items()])

    # Create an HTTPS connection to the CTFtime API
    conn = http.client.HTTPSConnection('ctftime.org')

    # Send a GET request to the API endpoint
    conn.request('GET', url + query_string)

    # Get the response from the API
    response = conn.getresponse()

    # Read the response body
    data = response.read().decode('utf-8')

    # Parse the JSON data
    return json.loads(data)

events = get_future_events(day_limit=days)
print(f"Retrieved {len(events)} CTFs for the next {days} days")



In [ ]:
def clean_events(events, restriction_list, useless_columns):
    # Converting list to dataframme
    events_df = pd.DataFrame(events)

    # Extract organizer named from organizer dictionary
    events_df['organizer_name'] = events_df['organizers'].apply(lambda x: x[0]['name'])

    # Computing the total duration of the event in hours and store it in a new column
    events_df['duration_hours'] = events_df['duration'].apply(lambda x: x['hours'] + x['days']*24)

    # Removing all onsite events
    events_df = events_df[events_df['onsite'] == False]

    # Keep only ctf where restictions are in $restriction_list
    events_df = events_df[events_df['restrictions'].apply(lambda x: x in restriction_list)]

    for restriction in restriction_list:
        # Add a boolean column named like the restriction that takes 1 if the value in restrictions is equal to the restriction
        events_df[restriction.lower()] = events_df['restrictions'].apply(lambda x: x == restriction)

    # Drop useless columns
    events_df = events_df.drop(useless_columns, axis=1)

    # Rename column id into ctftime_id
    events_df = events_df.rename(columns={'id': 'ctftime_id'})

    return events_df

events_df = clean_events(events=events, restriction_list=restriction_list, useless_columns=useless_columns)    
print(f"After data cleaning we found {events_df.shape[0]} interesting CTFs for you.")

In [ ]:
# Print all interesting events
events_df

In [ ]:
def save_2_xl(events_df, artifact_location):
    # Get current working directory
    cwd = os.getcwd()
    # Concatenate the final file path
    file_path = cwd+artifact_location
    # Save DataFrame to Excel file
    events_df.to_excel(file_path, index=False)
    
    return file_path

file_path = save_2_xl(events_df=events_df, artifact_location=artifact_location)
print(f"Future events stored in CSV file {file_path}")